### Exp: 1 working with Non-snapshot

Assumption: every same vehicle in the same parking timespan have a customer that belongs to that car

Vehicle : check ,same type, same cam , timestamp +-2 x,y is near each other ~~appear at least 2 timeframe~~ on the same x,y on non-snapshot cam

OUTPUT : vehicle_index : {timestamp range, x, y, vehicle type, }

camera used : mbk-14-12-vehicle 

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger.Approch1 import match_cross_to_vehicle, match_reverse_to_vehicle

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

CURRENT_CAM = 'mbk-14-11'

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "mbk-tourist-vehicle-object-20240429.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "mbk-tourist-raw-cross-object-20240429.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "mbk-tourist-raw-reverse-object-20240429.csv")
)

df_vehicle = Preprocess.df_clean_vehicle(df_vehicle=df_vehicle)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM+'-vehicle')
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_SNAPSHOT)

# df_vehicle[conf.VEHICLE_INTEREST_SNAPSHOT]

#drop the one that lifetime is less than 4 seconds 
df_vehicle = df_vehicle.loc[
        (df_vehicle["lifetime"] > 4),
    ]

#create a timestamp_unix at the end of lifespan
df_vehicle["timestamp_unix_end"] = (df_vehicle["timestamp_unix"] + (df_vehicle["lifetime"] *1.2))
# print(df_vehicle[[*conf.VEHICLE_INTEREST_SNAPSHOT,'timestamp_unix','timestamp_unix_end']].to_string())


/home/jeans/internship/parking-customer-count


## Cross vs Vehicle

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

In [3]:
def match_cross_to_vehicle(df_cross, df_vehicle, start_padding: int = 5, stop_padding: int = 20):
    
    df_vehicle_copy = df_vehicle.copy()
    df_vehicle_copy['cross_count'] = 0
    
    # Iterate over each row in df_cross
    for index, row in df_cross.iterrows():
        # Find the rows in df_vehicle where the timestamp is within the range
        mask = (df_vehicle_copy['timestamp_unix']-start_padding <= row['timestamp_unix']) & (df_vehicle_copy['timestamp_unix_end']+stop_padding >= row['timestamp_unix'])
        vehicle_rows = df_vehicle_copy[mask]
        if vehicle_rows.empty:
            # print('empty mask')
            continue
        # print(vehicle_rows.index.size)
        # if vehicle_rows.index.size > 1:
        #     print(vehicle_rows.to_string())
        
        # Calculate the distance from the current cross to each vehicle
        distances = np.sqrt((vehicle_rows['xmax'] - row['xmin'])**2 + (vehicle_rows['ymax'] - row['ymax'])**2)
        # # Find the index of the nearest vehicle
        nearest_index = distances.idxmin(axis=0)
        
        # Increment the cross count and mark as used for the nearest vehicle
        df_vehicle_copy.loc[nearest_index, 'cross_count'] += 1
    
    return df_vehicle_copy

In [4]:
# df_vehicle_cross_count = match_cross_to_vehicle(df_cross=df_cross, df_vehicle=df_vehicle)
# df_vehicle_cross_count[[*conf.VEHICLE_INTEREST_SNAPSHOT, 'cross_count']]

## Reverse vs Vehicle

In [5]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

In [6]:
def match_reverse_to_vehicle(df_reverse, df_vehicle, start_padding: int = 20, stop_padding: int = 5):
    
    df_vehicle_copy = df_vehicle.copy()
    df_vehicle_copy['reverse_count'] = 0
    
    # Iterate over each row in df_reverse
    for index, row in df_reverse.iterrows():
        # Find the rows in df_vehicle where the timestamp is within the range
        # print(row)
        mask = (df_vehicle_copy['timestamp_unix']-start_padding <= row['timestamp_unix']) & (df_vehicle_copy['timestamp_unix_end']+stop_padding >= row['timestamp_unix'])
        vehicle_rows = df_vehicle_copy[mask]
        if vehicle_rows.empty:
            # print('empty mask')
            continue
        # print(vehicle_rows.index.size)
        # if vehicle_rows.index.size > 1:
        #     print(vehicle_rows.to_string())
        
        # Calculate the distance from the current cross to each vehicle
        distances = np.sqrt((vehicle_rows['xmax'] - row['xmin'])**2 + (vehicle_rows['ymax'] - row['ymax'])**2)
        # # Find the index of the nearest vehicle
        nearest_index = distances.idxmin(axis=0)
        
        # Increment the cross count and mark as used for the nearest vehicle
        df_vehicle_copy.loc[nearest_index, 'reverse_count'] += 1
    
    return df_vehicle_copy

In [7]:
# df_vehicle_reverse_count = match_reverse_to_vehicle(df_reverse=df_reverse, df_vehicle=df_vehicle)
# df_vehicle_reverse_count[[*conf.VEHICLE_INTEREST_SNAPSHOT, 'timestamp_unix_end', 'reverse_count']]
#1714391893

## TOGETHER


In [8]:
df_vehicle = match_cross_to_vehicle(df_cross,df_vehicle)
df_vehicle = match_reverse_to_vehicle(df_reverse,df_vehicle)

df_vehicle[[*conf.CUSTOMER_INTEREST_SNAPSHOT, 'cross_count', 'reverse_count']]

,camera,timestamp_precise,xmin,xmax,ymin,ymax,cross_count,reverse_count
1245,mbk-14-11-vehicle,2024-04-29 06:43:16.470,1269,2307,0,412,8,2
1214,mbk-14-11-vehicle,2024-04-29 07:12:35.322,1159,2295,0,407,2,0
1216,mbk-14-11-vehicle,2024-04-29 07:13:02.794,1611,2283,0,440,0,0
1129,mbk-14-11-vehicle,2024-04-29 07:42:44.359,1595,2288,0,435,0,2
1130,mbk-14-11-vehicle,2024-04-29 07:43:00.595,939,2134,2,374,0,0
1109,mbk-14-11-vehicle,2024-04-29 07:45:01.125,1365,2178,0,412,1,3
1093,mbk-14-11-vehicle,2024-04-29 07:49:54.923,965,1960,0,379,0,0
1094,mbk-14-11-vehicle,2024-04-29 07:51:22.664,1098,1970,0,374,0,0
1096,mbk-14-11-vehicle,2024-04-29 07:53:31.746,1578,2267,0,440,0,0
952,mbk-14-11-vehicle,2024-04-29 08:42:31.083,1466,2166,0,416,7,2
